# Deep Neural Network For MNIST Classification
The dataset is called MNIST and refer to handwritten digit recogintion.

The dataset provides 70,000 images (28x28) of handwritten digits (1 digit per image).

The goal is to write an algo that detects which digit is written . Since there are only 10 digit(0,1,2,3,4,5,6,7,8,9),
this is a Classification problem with 10 classes.

Our goal is to build a neural network with 2 hidden layers.


## import the relevant librarys

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## LOad Data

In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\yeeme\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [4]:
# Split training and Testing data

mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [5]:
#Create Validation Sample

num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_sample=mnist_info.splits['train'].num_examples
num_test_sample=tf.cast(num_test_sample,tf.int64)


In [6]:
# SCaling of data

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image /= 255
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)

test_data=mnist_test.map(scale)

In [7]:
# Shuffling(keeping the same Information but in different Order)

BUFFER_SIZE=10000

shuffled_train_and_validation_data= scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [8]:
# Batching THe Data

BATCH_SIZE=100

train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_sample)

validation_inputs,validation_targets=next(iter(validation_data))

# Model


## Outline the model

In [9]:
input_size=784
output_size=10
hidden_layer_size=100

model=tf.keras.Sequential([
    
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
    ])



## Choose the Optimizer and the loss function

In [10]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Training

In [11]:
NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2 )

Epoch 1/5
540/540 - 4s - loss: 0.3372 - accuracy: 0.9045 - val_loss: 0.1723 - val_accuracy: 0.9537 - 4s/epoch - 8ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1418 - accuracy: 0.9590 - val_loss: 0.1150 - val_accuracy: 0.9695 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0986 - accuracy: 0.9711 - val_loss: 0.0915 - val_accuracy: 0.9735 - 3s/epoch - 5ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0754 - accuracy: 0.9771 - val_loss: 0.0776 - val_accuracy: 0.9777 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0604 - accuracy: 0.9810 - val_loss: 0.0689 - val_accuracy: 0.9803 - 3s/epoch - 5ms/step


## Test the Model

In [12]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 1s 572ms/step - loss: 0.0896 - accuracy: 0.9729


In [13]:
print('Test Loss:{0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

Test Loss:0.09. Test accuracy: 97.29%
